In [1]:
# 231115
# script for github

In [1]:
import pandas as pd

In [2]:
# from: https://colab.research.google.com/github/zaneveld/full_spectrum_bioinformatics/blob/master/content/06_biological_sequences/reading_and_writing_fasta_files.ipynb#scrollTo=ZkpJZr86_9Jz
def fastaDictionary(FastaFile):
    fasta = open(FastaFile, "r")
    """Return a dict of {id:gene_seq} pairs based on the sequences in the input FASTA file
    input_file -- a file handle for an input fasta file
    """
    parsed_seqs = {}
    curr_seq_id = None
    curr_seq = []

    for line in fasta:
        line = line.strip()
        
        #set up the dictionary
        if line.startswith(">"):
            # before going on to the next sequence in the file, join the final sequences together in the dictionary
            if curr_seq_id is not None:
                parsed_seqs[curr_seq_id] = ''.join(curr_seq)
                
            # rename/update/ curr_seq_id if there is a new line with ">"
            curr_seq_id = line[1:]
            # reset the sequence list to join
            curr_seq = []
            continue
        
        #add the sequence to the list
        curr_seq.append(line)

    #Add the final sequence to the dict
    parsed_seqs[curr_seq_id] = ''.join(curr_seq)
    return(parsed_seqs)

In [3]:
# extract sequences from Chi, Sal, and Tar
def extract_ChiSalTar(FastaFile, targetGene, outputFasta):
    fastaDict = fastaDictionary(FastaFile)
    if targetGene in fastaDict.keys():
        #print(targetGene)
        #print(fastaDict[targetGene])
        newFasta = open(outputFasta, "a+")
        newFasta.write(">"+targetGene)
        newFasta.write("\n")
        newFasta.write(fastaDict[targetGene])
        newFasta.write("\n")
        newFasta.close()

In [4]:
# extract sequences from Ath, Csa
def extract_AthCsa(FastaFile, targetGene, outputFasta):
    fastaDict = fastaDictionary(FastaFile)
    for gene_id, seq in fastaDict.items():
        #print(gene_id)
        #print(seq)
        geneIDlist = gene_id.strip().split(" ")
        fullName = geneIDlist[0].split(".")
        transcriptName = fullName[0] + "."+ fullName[1]
        if targetGene == transcriptName:
            newFasta = open(outputFasta, "a+")
            newFasta.write(">"+targetGene)
            newFasta.write("\n")
            newFasta.write(seq)
            newFasta.write("\n")
            newFasta.close()

In [5]:
# extract sequences from Aar, Bra, Cru
def extract_AarBraCru(FastaFile, targetGene, outputFasta):
    fastaDict = fastaDictionary(FastaFile)
    for gene_id, seq in fastaDict.items():
        #print(gene_id)
        #print(seq)
        geneIDlist = gene_id.strip().split(" ")
        fullName = geneIDlist[0]
        if targetGene == fullName:
            newFasta = open(outputFasta, "a+")
            newFasta.write(">"+targetGene)
            newFasta.write("\n")
            newFasta.write(seq)
            newFasta.write("\n")
            newFasta.close()

In [6]:
# list of genes to extract from target list
def ATgeneList(ATgeneList_path, OG_path):
    
    # open inclusive orthogroup file
    OGdf = pd.read_csv(OG_path)
    OGdf = OGdf.fillna("*")
    
    # create dictionary from OG list
    OG_Dict = {}
    for index, row in OGdf.iterrows():
        atGene = row[1]
        if atGene not in OG_Dict:
            OG_Dict[atGene] = []
            for i in range(2,10):
                if row[i] != "*":
                    geneList = row[i].split(", ")
                    for gene in geneList:
                        OG_Dict[atGene].append(gene)
    
    # open target gene list
    ATgeneList = open(ATgeneList_path, "r")
    geneOGList = []
    for gene in ATgeneList:
        gene = gene.strip()
        if gene in OG_Dict:
            targetGeneList = OG_Dict[gene]
            geneOGList.append([gene, targetGeneList])
    
    return(geneOGList)

In [9]:
# run script to extract sequences
main_path = r"/path_to_fasta"
# path to protein FASTA files
AarFastaFile = main_path + "/Ae.arabicum_v3.1_proteins_noTE_23160.fasta"
AthFastaFile = main_path + "/Athaliana_447_Araport11.protein_primaryTranscriptOnly.fa"
BraFastaFile = main_path + "/BrapaFPsc_277_v1.3.protein_primaryTranscriptOnly.fa"
ChiFastaFile = main_path + "/Chirsuta.primary.pep.fa"
CruFastaFile = main_path + "/Crubella_474_v1.1.protein_primaryTranscriptOnly.fa"
CsaFastaFile = main_path + "/Csativa.primary.pep.mod.fa"
SalFastaFile = main_path + "/Sal.pep"
TarFastaFile = main_path + "/Tar_v2_protein_geneID.fa"

# path to inclusive OG file and target gene list (AT gene IDs)
inclusiveOG_path = r"/path_to_inclusiveOG/inclusiveOG_fullSet_230928.csv"
ATtargets_path = r"/path_to_YABBY_list/AT_YABBY_geneID.txt"

#list of sequences - from a function
listOfTargets = ATgeneList(ATtargets_path, inclusiveOG_path)

for geneSet in listOfTargets:
    targetGene = geneSet[0]
    OGgenes = geneSet[1]
    
    #output file path
    ProteinSeqFasta_path = r"/path_to_out/"+targetGene+"_sequence.fa"    
    ALL_YABBY_Fasta_path = r"/path_to_out/ALL_sequence.fa"
    
    for gene in OGgenes:
        if gene.startswith("Aa") == True:
            extract_AarBraCru(AarFastaFile, gene, ProteinSeqFasta_path)
            extract_AarBraCru(AarFastaFile, gene, ALL_YABBY_Fasta_path)
        elif gene.startswith("AT") == True:
            extract_AthCsa(AthFastaFile, gene, ProteinSeqFasta_path)
            extract_AthCsa(AthFastaFile, gene, ALL_YABBY_Fasta_path)
        elif gene.startswith("Bra") == True:
            extract_AarBraCru(BraFastaFile, gene, ProteinSeqFasta_path)
            extract_AarBraCru(BraFastaFile, gene, ALL_YABBY_Fasta_path)
        elif gene.startswith("CARH") == True:
            extract_ChiSalTar(ChiFastaFile, gene, ProteinSeqFasta_path)
            extract_ChiSalTar(ChiFastaFile, gene, ALL_YABBY_Fasta_path)
        elif gene.startswith("Csa") == True:
            extract_AthCsa(CsaFastaFile, gene, ProteinSeqFasta_path)
            extract_AthCsa(CsaFastaFile, gene, ALL_YABBY_Fasta_path)
        elif gene.startswith("Carub") == True:
            extract_AarBraCru(CruFastaFile, gene, ProteinSeqFasta_path)
            extract_AarBraCru(CruFastaFile, gene, ALL_YABBY_Fasta_path)
        elif gene.startswith("Sal") == True:
            extract_ChiSalTar(SalFastaFile, gene, ProteinSeqFasta_path)
            extract_ChiSalTar(SalFastaFile, gene, ALL_YABBY_Fasta_path)
        elif gene.startswith("gene") == True:
            extract_ChiSalTar(TarFastaFile, gene, ProteinSeqFasta_path)
            extract_ChiSalTar(TarFastaFile, gene, ALL_YABBY_Fasta_path)